<img src="../ancillarydata/logos/LundUniversity_C2line_RGB.png" width="150" align="left"/>
<br>
<img src="../ancillarydata/logos/Icos_Logo_CMYK_Regular_SMpng.png" width="327" align="right"/>
<br>
<a id='introduction'></a>
<a id='modules'></a>

<br>

# <font color=#B98F57>PhD course</font>
### <font color=#000083>From CO$_2$ in situ measurements to carbon balance maps as a tool to support national carbon accounting</font>

<br>

**Organized by:** Lund University dept. of Physical Geography & Ecosystem Science and ICOS ERIC <br>
**Funded by:**    Lund University dept. of Physical Geography & Ecosystem Science, ICOS ERIC and ClimBEco <br>
**Course dates:** March 9th 2020 - March 13th 2020 <br>
**Location:** Lund, Sweden

**Notebook developed by:** Karolina Pantazatou, Ute Karstens & Claudio D' Onofrio

<br>
<br>

# Notebook including Python Functions for ClimBEco course
This notebook includes ready functions or code snippets for reading, processing and plotting data related to the 2020 ClimBEco course.

The notebook is divided in different sections:

- [Python Modules](#modules)
- [FLUXNET data](#fluxnet)
- [LPJ-GUESS (Paul's dataset)](#lpjguess_paul)
- [LPJ-GUESS map data](#lpjguess_map)
- [LUMIA data](#lumia)
- [EDAGR data](#edgar)
- [Extract Timeseries from map](#tsmaps)
- [Country Masks](#masks)
- [Station class](#stclass)

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return false;}

## Python modules
List the Python modules used for the functions and/or code snippets

In [ ]:
#Import modules
import os
import numpy as np
import pandas as pd
from pandas.tseries.frequencies import to_offset
import folium
import geopandas as gpd
import netCDF4 as cdf
from datetime import datetime, timedelta
from ipywidgets import Dropdown, Button, HBox, VBox, Output, RadioButtons, Checkbox, FloatText, interact_manual
from IPython.display import display, clear_output, HTML

#Import plotting modules from cartopy:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.feature import NaturalEarthFeature, LAND, COASTLINE, LAKES

#Import plotting modules from matplotlib:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker


#Import Bokeh modules:
from bokeh.models import Plot, LinearAxis, Grid, BasicTicker, HoverTool, ColorBar, ColumnDataSource, Label, Legend, GeoJSONDataSource
from bokeh.plotting import figure
from bokeh.layouts import column
import bokeh.io
from bokeh.io import show, output_notebook, reset_output, push_notebook

#Turn off warnings:
import warnings
warnings.filterwarnings('ignore')

#Set the notebook as the default output location:
bokeh.io.reset_output()
bokeh.io.output_notebook()


## Paths

In [ ]:
personal_home = '/data/project/'
path_climbeco = personal_home+'/climbeco/data/'
path_fluxnet_dd = path_climbeco+"/fluxnet/obs_dd/"
path_fluxnet_hh = path_climbeco+"/fluxnet/obs_hh/"
path_masks = path_climbeco+'masks/'
path_lumia = path_climbeco+'lumia/'
path_lpjmaps = path_climbeco+'lpjguess_maps/'
path_edgar = path_climbeco+'edgar/'
path_lpj = path_climbeco + 'lpjguess/lpj_clipped_2015_2018/'
path_lpj_conif = path_lpj +'Conif_output/'
path_lpj_decid = path_lpj +'Decid_output/'
path_lpj_grass_wet_crop = path_lpj + 'Grasswetcrop_output/'
path_lpj_pnv = path_lpj + 'PNV_output_allsites/'
path_geo_data = path_climbeco + 'geo_data/'
path_geo_natural_earth10m = path_geo_data + 'ne_10m_admin_0_countries/'

<div style="text-align: right"><a href="#introduction">Back to top</a></div>

<a id='fluxnet'></a>

## FLUXNET

### Dictionaries 

This part includes the dictionaries created for:

- climbeco stations 
    - There are 18 stations in total
    - The dictionary includes summarized info per station (i.e. station name, country, country code, station type, latitude, longitude). 
    - The keys of the dictionary are the station code (3-character string).


- FLUXNET variables (daily values dataset)
    - The keys are the FLUXNET variable codes, included as column-names in the FLUXNET dataset.
    - The dictionary values include the description of the variable code and the variable unit.


- FLUXNET variables (half-hourly values dataset)
    - The keys are the FLUXNET variable codes, included as column-names in the FLUXNET dataset.
    - The dictionary values include the description of the variable code and the variable unit.
    
    
<font color='red'>There might be differences in the types of variables or variable units between the FLUXNET half-hourly dataset and the FLUXNET daily-values dataset.</font>

In [ ]:
#Create dictionary for stations:
#Every station code is connected to a list, incl.
#station name, country, country code, station type, latitude, longitude, url
#lpj-guess latitude, lpj-guess longitude.
stations_dict = {"Sor":["Soroe", "Denmark", "DK","Deciduous forest", 55.48587, 11.64464,
                        "http://www.europe-fluxdata.eu/home/site-details?id=DK-Sor",
                        55.25, 11.75],
                 "Let":["Lettosuo", "Finland", "FI", "Peatland", 60.64183, 23.95952,
                        "http://www.europe-fluxdata.eu/home/site-details?id=FI-Let",
                        60.75, 23.75],
                 "Sii":["Siikaneva", "Finland", "FI", "Fen", 61.83265, 24.19285,
                        "http://www.europe-fluxdata.eu/home/site-details?id=FI-Sii",
                        61.75, 24.25],
                 "Akm":["Anklam", "Germany", "DE", "Wetland", 53.86617, 13.68342,
                        "http://www.europe-fluxdata.eu/home/site-details?id=DE-Akm",
                        53.75, 13.75],
                 "Geb":["Gebesee", "Germany", "DE", "Cropland", 51.09974, 10.91462,
                        "http://www.europe-fluxdata.eu/home/site-details?id=DE-Geb",
                        51.25, 10.75],
                 "Gri":["Grillenburg", "Germany", "DE", "Grassland", 50.95004, 13.51259,
                        "http://www.europe-fluxdata.eu/home/site-details?id=DE-Gri",
                        50.75, 13.75],
                 "Hai":["Hainich", "Germany", "DE", "Deciduous forest", 51.07921, 10.45216,
                        "http://www.europe-fluxdata.eu/home/site-details?id=DE-Hai",
                        51.25, 10.25],
                 "HoH":["Hohes Holz", "Germany", "DE", "Deciduous forest", 52.08656, 11.22235,
                        "http://www.europe-fluxdata.eu/home/site-details?id=DE-HoH",
                        52.25, 11.25],
                 "Hte":["Huetelmoor", "Germany", "DE", "Wetland", 54.210278, 12.176111,
                        "http://www.europe-fluxdata.eu/home/site-details?id=DE-Hte",
                        54.25, 12.25],
                 "RuR":["Rollesbroich", "Germany", "DE", "Grassland", 50.6219142, 6.3041256,
                        "http://www.europe-fluxdata.eu/home/site-details?id=DE-RuR",
                        50.75, 6.25],
                 "RuS":["Selhausen Juelich", "Germany", "DE", "Cropland", 50.86589, 6.44712,
                        "http://www.europe-fluxdata.eu/home/site-details?id=DE-RuS",
                        50.75, 6.25],
                 "Loo":["Loobos", "Netherlands", "NL", "Coniferous forest", 52.16648, 5.74355,
                        "http://www.europe-fluxdata.eu/home/site-details?id=NL-Loo",
                        52.25, 5.75],
                 "Deg":["Degero", "Sweden", "SE", "Wetland", 64.18203, 19.55654,
                        "http://www.europe-fluxdata.eu/home/site-details?id=SE-Deg",
                        64.25, 19.75],
                 "Htm":["Hyltemossa", "Sweden", "SE", "Coniferous forest", 56.09763, 13.41897,
                        "http://www.europe-fluxdata.eu/home/site-details?id=SE-Htm",
                        56.25, 13.25],
                 "Lnn":["Lanna", "Sweden", "SE", "Cropland", 58.34063, 13.10177,
                        "http://www.europe-fluxdata.eu/home/site-details?id=SE-Lnn",
                        58.25, 13.25],
                 "Nor":["Norunda", "Sweden", "SE", "Coniferous forest", 60.08644, 17.47946,
                        "http://www.europe-fluxdata.eu/home/site-details?id=SE-Nor",
                        60.25, 17.25],
                 "Ros":["Rosinedal", "Sweden", "SE", "Coniferous forest", 64.1725, 19.738,
                        "http://www.europe-fluxdata.eu/home/site-details?id=SE-Ros",
                        64.25, 19.75],
                 "Svb":["Svartberget", "Sweden", "SE", "Coniferous forest", 64.25609, 19.77457,
                        "http://www.europe-fluxdata.eu/home/site-details?id=SE-Svb",
                        64.25, 19.75]}

In [ ]:
#Create variable dictionary for daily values:
variables_dd_dict = {"TA_F":["Air temperature", "deg C"],
                     "TA_F_NIGHT":["Average air temperature (nighttime)", "deg C"],
                     "TA_F_DAY":["Average air temperature (daytime)", "deg C"],
                     "SW_IN_F":["Shortwave incoming radiation", "W m-2"],
                     "LW_IN_JSB_F":["Longwave incoming radiation", "W m-2"],
                     "VPD_F":["Vapor Pressure Deficit", "hPa"],
                     "PA_F":["Atmospheric pressure", "kPa"],
                     "P_F":["Precipitation", "mm"],
                     "P_F_QC":["Precipitation quality flag", "adimensional",
                               "fraction between 0-1, indicating percentage of measured data"],
                     "WS_F":["Wind speed", "m s-1"],
                     "CO2_F_MDS":["CO2 mole fraction, gap-filled with MDS", "umolCO2 mol-1"],
                     "TS_F_MDS_1":["Soil temperature (index=1)", "deg C"],
                     "TS_F_MDS_2":["Soil temperature (index=2)", "deg C"],
                     "TS_F_MDS_3":["Soil temperature (index=3)", "deg C"],
                     "TS_F_MDS_4":["Soil temperature (index=4)", "deg C"],
                     "TS_F_MDS_5":["Soil temperature (index=5)", "deg C"],
                     "TS_F_MDS_6":["Soil temperature (index=6)", "deg C"],
                     "TS_F_MDS_7":["Soil temperature (index=7)", "deg C"],
                     "SWC_F_MDS_1":["Soil water content (index=1)", "%"],
                     "SWC_F_MDS_2":["Soil water content (index=2)", "%"],
                     "SWC_F_MDS_3":["Soil water content (index=3)", "%"],
                     "SWC_F_MDS_4":["Soil water content (index=4)", "%"],
                     "SWC_F_MDS_5":["Soil water content (index=5)", "%"],
                     "SWC_F_MDS_6":["Soil water content (index=6)", "%"],
                     "SWC_F_MDS_7":["Soil water content (index=7)", "%"],
                     "SWC_F_MDS_1_QC":["Soil water content quality flag (index=1)", "adimensional",
                                       "fraction between 0-1, indicating percentage of measured and good quality gapfill data"],
                     "SWC_F_MDS_2_QC":["Soil water content quality flag (index=2)", "adimensional",
                                       "fraction between 0-1, indicating percentage of measured and good quality gapfill data"],
                     "SWC_F_MDS_3_QC":["Soil water content quality flag (index=3)", "adimensional",
                                       "fraction between 0-1, indicating percentage of measured and good quality gapfill data"],
                     "SWC_F_MDS_4_QC":["Soil water content quality flag (index=4)", "adimensional",
                                       "fraction between 0-1, indicating percentage of measured and good quality gapfill data"],
                     "SWC_F_MDS_5_QC":["Soil water content quality flag (index=5)", "adimensional",
                                       "fraction between 0-1, indicating percentage of measured and good quality gapfill data"],
                     "SWC_F_MDS_6_QC":["Soil water content quality flag (index=6)", "adimensional",
                                       "fraction between 0-1, indicating percentage of measured and good quality gapfill data"],
                     "SWC_F_MDS_7_QC":["Soil water content quality flag (index=7)", "adimensional",
                                       "fraction between 0-1, indicating percentage of measured and good quality gapfill data"],
                     "G_F_MDS":["Soil heat flux", "W m-2"],
                     "LE_CORR":["Latent heat flux", "W m-2"],
                     "LE_CORR_JOINTUNC":["Joint uncertainty estimation for latent heat flux", "W m-2"],
                     "H_CORR":["Sensible heat flux", "W m-2"],
                     "H_CORR_JOINTUNC":["Joint uncertainty estimation for sensible heat flux", "W m-2"],
                     "NIGHT_RANDUNC_N":["Number of half hours classified as nighttime", "adimensional", 
                                        "number of half-hours"],
                     "DAY_RANDUNC_N":["Number of half hours classified as daytime", "adimensional",
                                      "number of half-hours"],
                     "GPP_DT_VUT_MEAN":["Gross Primary Production (daytime)", "gC m-2 d-1"],
                     "GPP_DT_VUT_SE":["Standard Error for Gross Primary Production (daytime)", "gC m-2 d-1"],
                     "GPP_NT_VUT_MEAN":["Gross Primary Production (nighttime)", "gC m-2 d-1"],
                     "GPP_NT_VUT_SE":["Standard Error for Gross Primary Production (nighttime)", "gC m-2 d-1"],
                     "RECO_DT_VUT_MEAN":["Ecosystem Respiration (daytime)", "gC m-2 d-1"],
                     "RECO_DT_VUT_SE":["Standard Error for Ecosystem Respiration (daytime)", "gC m-2 d-1"],
                     "RECO_NT_VUT_MEAN":["Ecosystem Respiration (nighttime)", "gC m-2 d-1"],
                     "RECO_NT_VUT_SE":["Standard Error for Ecosystem Respiration (nighttime)", "gC m-2 d-1"],
                     "NEE_VUT_USTAR50":["Net Ecosystem Exchange (USTAR50)", "gC m-2 d-1"],
                     "NEE_VUT_USTAR50_QC":["Quality flag for Net Ecosystem Exchange (USTAR50)", "adimensional",
                                           "fraction between 0-1, indicating percentage of measured and good quality gapfill data"],
                     "NEE_VUT_MEAN":["Net Ecosystem Exchange (VUT MEAN)", "gC m-2 d-1"],
                     "NEE_VUT_MEAN_QC":["Quality flag for Net Ecosystem Exchange (VUT MEAN)", "adimensional",
                                        "average of 40 NEE_VUT_XX_QC for the period"],
                     "NEE_VUT_SE":["Standard Error for Net Ecosystem Exchange (VUT MEAN)", "gC m-2 d-1"],
                     "NEE_VUT_REF_NIGHT":["Average Net Ecosystem Exchange (nighttime)", "umolCO2 m-2 s-1"],
                     "NEE_VUT_REF_NIGHT_SD":["Standard Deviation for Net Ecosystem Exchange (nighttime)", "umolCO2 m-2 s-1"],
                     "NEE_VUT_REF_NIGHT_QC":["Quality flag for Net Ecosystem Exchange (nighttime)", "adimensional",
                                             "fraction between 0-1, indicating percentage of measured and good quality gapfill data"],
                     "NEE_VUT_REF_DAY":["Average Net Ecosystem Exchange (daytime)", "umolCO2 m-2 s-1"],
                     "NEE_VUT_REF_DAY_SD":["Standard Deviation for Net Ecosystem Exchange (daytime)", "umolCO2 m-2 s-1"],
                     "NEE_VUT_REF_DAY_QC":["Quality flag for Net Ecosystem Exchange (daytime)", "adimensional",
                                           "fraction between 0-1, indicating percentage of measured and good quality gapfill data"]}


In [ ]:
#Create variable dictionary for half-hourly values:
variables_hh_dict = {"TA_F":["Air temperature", "deg C"],
                     "SW_IN_F":["Shortwave incoming radiation", "W m-2"],
                     "LW_IN_JSB_F":["Longwave incoming radiation", "W m-2"],
                     "VPD_F":["Vapor Pressure Deficit", "hPa"],
                     "PA_F":["Atmospheric pressure", "kPa"],
                     "P_F":["Precipitation", "mm"],
                     "P_F_QC":["Precipitation quality flag", "adimensional",
                               "0 = measured; 2 = downscaled from ERA"],
                     "WS_F":["Wind speed", "m s-1"],
                     "CO2_F_MDS":["CO2 mole fraction, gap-filled with MDS", "umolCO2 mol-1"],
                     "TS_F_MDS_1":["Soil temperature (index=1)", "deg C"],
                     "TS_F_MDS_2":["Soil temperature (index=2)", "deg C"],
                     "TS_F_MDS_3":["Soil temperature (index=3)", "deg C"],
                     "TS_F_MDS_4":["Soil temperature (index=4)", "deg C"],
                     "TS_F_MDS_5":["Soil temperature (index=5)", "deg C"],
                     "TS_F_MDS_6":["Soil temperature (index=6)", "deg C"],
                     "TS_F_MDS_7":["Soil temperature (index=7)", "deg C"],
                     "SWC_F_MDS_1":["Soil water content (index=1)", "%"],
                     "SWC_F_MDS_2":["Soil water content (index=2)", "%"],
                     "SWC_F_MDS_3":["Soil water content (index=3)", "%"],
                     "SWC_F_MDS_4":["Soil water content (index=4)", "%"],
                     "SWC_F_MDS_5":["Soil water content (index=5)", "%"],
                     "SWC_F_MDS_6":["Soil water content (index=6)", "%"],
                     "SWC_F_MDS_7":["Soil water content (index=7)", "%"],
                     "SWC_F_MDS_1_QC":["Soil water content quality flag (index=1)", "adimensional",
                                       "0 = measured; 1 = good quality gapfill; 2 = medium; 3 = poor"],
                     "SWC_F_MDS_2_QC":["Soil water content quality flag (index=2)", "adimensional",
                                       "0 = measured; 1 = good quality gapfill; 2 = medium; 3 = poor"],
                     "SWC_F_MDS_3_QC":["Soil water content quality flag (index=3)", "adimensional",
                                       "0 = measured; 1 = good quality gapfill; 2 = medium; 3 = poor"],
                     "SWC_F_MDS_4_QC":["Soil water content quality flag (index=4)", "adimensional",
                                       "0 = measured; 1 = good quality gapfill; 2 = medium; 3 = poor"],
                     "SWC_F_MDS_5_QC":["Soil water content quality flag (index=5)", "adimensional",
                                       "0 = measured; 1 = good quality gapfill; 2 = medium; 3 = poor"],
                     "SWC_F_MDS_6_QC":["Soil water content quality flag (index=6)", "adimensional",
                                       "0 = measured; 1 = good quality gapfill; 2 = medium; 3 = poor"],
                     "SWC_F_MDS_7_QC":["Soil water content quality flag (index=7)", "adimensional",
                                       "0 = measured; 1 = good quality gapfill; 2 = medium; 3 = poor"],
                     "G_F_MDS":["Soil heat flux", "W m-2"],
                     "LE_CORR":["Latent heat flux", "W m-2"],
                     "LE_CORR_JOINTUNC":["Joint uncertainty estimation for latent heat flux", "W m-2"],
                     "H_CORR":["Sensible heat flux", "W m-2"],
                     "H_CORR_JOINTUNC":["Joint uncertainty estimation for sensible heat flux", "W m-2"],
                     "GPP_DT_VUT_MEAN":["Gross Primary Production (daytime)", "umolCO2 m-2 s-1"],
                     "GPP_DT_VUT_SE":["Standard Error for Gross Primary Production (daytime)", "umolCO2 m-2 s-1"],
                     "GPP_NT_VUT_MEAN":["Gross Primary Production (nighttime)", "umolCO2 m-2 s-1"],
                     "GPP_NT_VUT_SE":["Standard Error for Gross Primary Production (nighttime)", "umolCO2 m-2 s-1"],
                     "RECO_DT_VUT_MEAN":["Ecosystem Respiration (daytime)", "umolCO2 m-2 s-1"],
                     "RECO_DT_VUT_SE":["Standard Error for Ecosystem Respiration (daytime)", "umolCO2 m-2 s-1"],
                     "RECO_NT_VUT_MEAN":["Ecosystem Respiration (nighttime)", "umolCO2 m-2 s-1"],
                     "RECO_NT_VUT_SE":["Standard Error for Ecosystem Respiration (nighttime)", "umolCO2 m-2 s-1"],
                     "NEE_VUT_USTAR50":["Net Ecosystem Exchange (USTAR50)", "umolCO2 m-2 s-1"],
                     "NEE_VUT_USTAR50_QC":["Quality flag for Net Ecosystem Exchange (USTAR50)", "adimensional",
                                           "0 = measured; 1 = good quality gapfill; 2 = medium; 3 = poor"],
                     "NEE_VUT_MEAN":["Net Ecosystem Exchange (VUT MEAN)", "umolCO2 m-2 s-1"],
                     "NEE_VUT_MEAN_QC":["Quality flag for Net Ecosystem Exchange (VUT MEAN)", "adimensional",
                                        "average of percentages of good data (NEE_VUT_XX_QC is 0 or 1) from 40 NEE_VUT_XX_QC"],
                     "NEE_VUT_SE":["Standard Error for Net Ecosystem Exchange (VUT MEAN)", "umolCO2 m-2 s-1"]}


In [ ]:
lpj_var_dict = {'gpp':'dgpp.out',
                'npp':'dnpp.out',
                'nee':'dnee.out',
                'rtot':'drtot.out',
                'prec':'dprec.out',
                'swrad':'dswrad.out',
                'temp':'dtemp.out'}

In [ ]:
lpj_veg_type_dict = {'coniferous':path_lpj_conif,
                     'conif':path_lpj_conif,
                     'deciduous':path_lpj_decid,
                     'decid':path_lpj_decid,
                     'pnv':path_lpj_pnv,
                     'grass_wet_crop':path_lpj_grass_wet_crop}

In [ ]:
#Create dictionary to assign variable to FLUXNET variable name:
fluxnet_dict = {'NEE':'NEE_VUT_MEAN',
                'GPP':'GPP_DT_VUT_MEAN',
                'RTOT':'RECO_DT_VUT_MEAN'}

In [ ]:
#Create dictionary to assign station vegetation type to lpj output directory:
veg_type_to_lpjpath_dict = {'Coniferous forest':'conif',
                            'Cropland':'grass_wet_crop',
                            'Deciduous forest':'decid',
                            'Fen':'grass_wet_crop',
                            'Grassland':'grass_wet_crop',
                            'Peatland':'grass_wet_crop',
                            'Wetland':'grass_wet_crop'}

In [ ]:
#Create function to read FLUXNET-file with half-hourly values:
def read_fluxnet_hh(path, st_code):
      
    #Define file name:
    file = "FLX_"+stations_dict[st_code][2]+"-"+st_code+"_HH_2015_2018.csv"
    
    #Create full path to file:
    fullpath = path + file
    
    #Read csv to pandas dataframe:
    df = pd.read_csv(fullpath,
                     header = 0,
                     sep = ",",
                     parse_dates = ["TIMESTAMP_START", "TIMESTAMP_END"])

    #Return dataframe:
    return df.loc[df.NEE_VUT_MEAN_QC>0]


In [ ]:
#Create function to read FLUXNET-file with daily values:
def read_fluxnet_dd(path, st_code):
      
    #Define file name:
    file = "FLX_"+stations_dict[st_code][2]+"-"+st_code+"_DD_2015_2018.csv"
    
    #Create full path to file:
    fullpath = path + file
    
    #Read csv to pandas dataframe:
    df = pd.read_csv(fullpath,
                     header = 0,
                     sep = ",",
                     parse_dates = ["TIMESTAMP"])

    #Return dataframe:
    return df.loc[df.NEE_VUT_MEAN_QC>0]

In [ ]:
def plotmap_climbeco(stations_df, selected_station):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue Feb 04 10:40:00 2020
    Last Changed:     Tue Feb 04 10:40:00 2020
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes a dataframe containing info about ICOS Stations and the 3-character
                      station code of a selected station as input and returns an interactive Folium Map, with
                      the location of the selected station highlighted in red. 
                      Folium (URL): https://python-visualization.github.io/folium/quickstart.html
                      
    Input parameters: 1. Dataframe with Information regarding ICOS Stations
                         (var_name: 'stations_df', var_type: Pandas Dataframe)
                      2. Station 3-character Code
                         (var_name: 'selected_station', var_type: String)

    Output:           Folium Map (Folium Map Object)
    
    """
    
    #Import modules:
    import folium

    #Create folium map-object:
    m = folium.Map(
        location=[60.07921, 9.0000],
        zoom_start=4)

    #Add marker-tooltip:
    tooltip = 'Click to view station info'

    def add_marker(map_obj, station_code, marker_color):

        #Add popup text:
        popup=folium.Popup('Name: <b>'+stations_df.name.loc[stations_df.index==station_code].values[0]+'</b><br>'+
                           'Code: <b>'+station_code+'</b><br>'+
                           'Country: <b>'+stations_df.country.loc[stations_df.index==station_code].values[0]+'</b><br>'+
                           'Country code: <b>'+stations_df.country_code.loc[stations_df.index==station_code].values[0]+'</b><br>'+
                           'Vegetation: <b>'+stations_df.vegetation.loc[stations_df.index==station_code].values[0]+'</b><br>'+
                           'Latitude: <b>'+str(stations_df.lat.loc[stations_df.index==station_code].values[0])+'</b><br>'+
                           'Longitude: <b>'+str(stations_df.lon.loc[stations_df.index==station_code].values[0])+'</b><br>'+
                           'URL: <a href="'+stations_df.url.loc[stations_df.index==station_code].values[0]+
                           '"target="_blank">link</a>',
                           max_width=450)

        #Create marker and add it to the map:
        folium.Marker(location=[float(stations_df.lat.loc[stations_df.index==station_code].values[0]),
                                float(stations_df.lon.loc[stations_df.index==station_code].values[0])],
                      popup=popup,
                      icon=folium.Icon(color=marker_color, icon='leaf'),
                      tooltip=tooltip).add_to(map_obj)


    #Get list of stations (not incl. selected station):
    station_ls = [i for i in stations_df.index.values if i!=selected_station]  

    #Create markers for all stations except selected station:
    for st in station_ls:
        add_marker(m, st, 'green') 

    #Add marker for selected station:
    add_marker(m, selected_station, 'darkred')

    #Show map:
    display(m)



In [ ]:
def printmd(string):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri May 10 12:00:00 2019
    Last Changed:     Fri May 10 12:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that allows you to print the string input parameter using
                      markdown formatting code.
                      
    Input parameters: String of characters
                      (var_name: 'string', var_type: String)

    Output:           String     
    
    """
    
    #import module:
    from IPython.display import Markdown, display
    
    #Enable displaying string with markdown code:
    display(Markdown(string))

In [ ]:
#Create function to plot FLUXNET-variables:
def plot_fluxnet_var(df, var, st_name, var_dict, stations_dict, color):

    #Create a new plot with a datetime axis type:
    p = figure(plot_width=900, plot_height=500, x_axis_type="datetime")


    #Check if variable includes a adimensional quality-values:
    if(var_dict[var][1]!="adimensional"): 

        #Add renderers:
        r0 = p.circle(df[df.columns.values[0]].values, df[var].values, size=4, color=color, alpha=0.2)
        r1 = p.line(df[df.columns.values[0]].values, df[var].values, color=color)

        #Create legend:
        legend = Legend(items=[(var_dict[var][0] + " sample", [r0]),
                               (var_dict[var][0] + " continuous", [r1])], location= 'bottom_center')
        legend.click_policy='hide'


    else:

        #Add renderers:
        r0 = p.circle(df[df.columns.values[0]].values, df[var].values, size=4, color=color, alpha=0.2)

        #Create legend:
        legend = Legend(items=[(var_dict[var][0], [r0])], location= 'bottom_center')


    #Add tooltip:
    p.add_tools(HoverTool(tooltips=[('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
                                    (var,'@y{0.f}'),],
                          formatters={'@x'      : 'datetime',},
                          # visa ett tooltip när musen är i lodrätt-linje med motsvarande glyph
                          mode='vline'))  


    # NEW: customize by setting attributes
    p.title.text = var_dict[var][0] +': '+ stations_dict[st_name][0]+', '+stations_dict[st_name][1]+ " (2015 - 2018)"
    p.grid.grid_line_alpha = 0
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = var_dict[var][0] +"\n("+var_dict[var][1]+")"
    p.ygrid.band_fill_color = "olive"
    p.ygrid.band_fill_alpha = 0.1
    p.title.align = 'center'
    p.title.text_font_size = '13pt'
    p.title.offset = 15


    #Definiera font för x-axel och y-axel titlarna :
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units

    #Set legend format:
    legend.orientation = 'horizontal'
    legend.spacing = 10 #sets the distance between legend entries


    #Set location of copyright-label:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')    

    #Create copyright-label:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'

    #Add copyright-label to plot:
    p.add_layout(caption1, 'below')

    #Add legend to figure:
    p.add_layout(legend, 'below')

    #Deactivate hover-tool:
    p.toolbar.active_inspect = None

    
    #Return plot:
    return p

In [ ]:
#Create function to display widgets for plotting FLUXNET-files with daily-values:
def create_fluxnet_dd_widgets():

    #Import modules:
    from ipywidgets import Dropdown, Checkbox, ColorPicker, interact_manual

    #Create tuples for widgets labels:
    st_list = [(i[1][0], [i[0], i[1][0], i[1][1], i[1][2], i[1][3], i[1][4], i[1][5]])
               for i in stations_dict.items()]

    var_list = [(i[1][0], [i[0], i[1][0],  i[1][1]])
                for i in variables_dd_dict.items()]

    #Sort station list: 
    st_list.sort(reverse=False)
    var_list.sort(reverse=False)

    #Create widgets:
    stations = Dropdown(options = st_list)
    variables_dd = Dropdown(options = var_list)
    citation = Checkbox(value=True, description='Citation', disabled=False)
    colors = ColorPicker(concise=False, description='Plot color', value='navy', disabled=False)

    #Function to update plot:
    def update_plot_dd(Station, Variable, color, cit):

        #Read FLUXNET file:
        df = read_fluxnet_dd(path_fluxnet_dd, Station[0])

        #Check if variable is included in dataframe:
        if(Variable[0] in df.columns.values):

            #Call function to plot variable values:
            plot = plot_fluxnet_var(df, Variable[0], Station[0], variables_dd_dict, stations_dict, color)
            
            #Convert stations dictionary to pandas dataframe:
            stations_df = pd.DataFrame.from_dict(stations_dict,
                                                 orient='index',
                                                 columns=['name', 'country', 'country_code', 'vegetation', 'lat', 'lon', 'url', 'lpj_lat', 'lpj_lon'])

            #Show plot
            show(plot, notebook_handle=True)
            
            #If citation-checkbox is checked:
            if(cit):

                #Citation in APA-format:
                apa_cit = 'Drought 2018 Team & ICOS Ecosystem Thematic Centre (2020). Drought-2018 ecosystem eddy covariance flux product in FLUXNET-Archive format - release 2019-2 (Version 1.0). ICOS Carbon Portal. https://doi.org/10.18160/yvr0-4898'

                #Print citation title:
                print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')

                #Print APA-style citation:
                printmd("<sub>"+apa_cit+"</sub>")
                
                print()
                print()
            
            #Show map:
            plotmap_climbeco(stations_df, Station[0])
            


        else:

            print("\033[0;31;1m "+'There are no '+Variable[1]+'-values for '+Variable[0]+
                  " included in this file!"+"\033[0;31;0m\n\n")


        
            
            

    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_dd,
                                 Station = stations,
                                 Variable = variables_dd,
                                 color = colors,
                                 cit = citation)


    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '430px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 202px'
    interact_c.widget.children[1].layout.width = '430px'
    interact_c.widget.children[1].layout.margin = '2px 2px 2px 202px'
    interact_c.widget.children[2].layout.width = '430px'
    interact_c.widget.children[2].layout.margin = '2px 2px 2px 202px'
    interact_c.widget.children[3].layout.width = '430px'
    interact_c.widget.children[3].layout.margin = '2px 2px 2px 202px'
    interact_c.widget.children[4].description = 'Update Plot'
    interact_c.widget.children[4].button_style = 'danger'
    interact_c.widget.children[4].style.button_color = '#3973ac'
    interact_c.widget.children[4].layout.margin = '10px 10px 40px 385px' # top/right/bottom/left

<div style="text-align: right"><a href="#introduction">Back to top</a></div>
<a id='lpjguess_paul'></a>

## LPJ-GUESS (Paul's dataset)

In [ ]:
def read_lpjguess_st(vegetation, eco_variable, station, st_dict=stations_dict):
    
    '''
    Description: Function that reads LPJ-GUESS output for given station.
    '''
    
    path = lpj_veg_type_dict[vegetation.lower()]
    filename = lpj_var_dict[eco_variable.lower()]
    
    #Set offset:
    offset = 1460
    
    #Read file:
    df = pd.read_csv(path+filename,
                     header=0,
                     sep=',',
                     engine='python',
                     parse_dates=['time'])
    
    #Extract data for input station based on its corresponding LPJ-GUESS coordinates:
    st_df = df.loc[(df.Lon==stations_dict[station][8]) & (df.Lat==stations_dict[station][7])]
    
    
    #############################################################################
    #Some stations have been assigned with the same lpj-guess grid-coordinates.
    #The lpj-guess output does not include a station name column.
    #Lpj-guess output values can be related to a station based on coordinates.
    #The following code sepparates values for stations being assigned with the 
    #same coordinate. This is possible due to the order the output is produced in.
    #############################################################################
    
    #Check if output-files include PNV-data:
    if((path[-20:-17]=='PNV')):
        
        #Check if station is "Degerö":
        if(station=='Deg'):
            
            #
            st_df = st_df.iloc[:offset]
            
            
        elif(station=='Ros'):
            print('Svb PNV')
            
            st_df = st_df.iloc[offset:2*offset]
            
        elif(station=='Svb'):
            
            
            st_df = st_df.iloc[(2*offset):]
            
            
    elif((path[-20:-17]!='PNV')):
        
        if((station=='Ros') | (station=='RuR')):

            st_df = st_df.iloc[:offset]

        if((station=='Svb') | (station=='RuS')):

            st_df = st_df.iloc[offset:]
    
    #Add entry for the 29th of Feb 2016 (Leap Year)
    #Same value as 28th of Feb
    listOfSeries = [pd.Series([st_df.Lon.iloc[0], st_df.Lon.iloc[0], 2016, 60, st_df.Value.loc[st_df.time==datetime(2016,2,28)].values[0], datetime.strptime(str(2016)+' '+str(60), '%Y %j')], index=st_df.columns)]
    
    #Pass a list of series to the append() to add multiple rows
    mod_df = st_df.append(listOfSeries , ignore_index=True)
    
    #Sort values by the column "time":
    mod_df.sort_values(by=['time'])
    
    #Return dataframe:
    return mod_df

In [ ]:
#Function that calculates monthly values for a certain column of a pandas dataframe.
#The dataframe should have a datetime index:
def calc_monthly_values(df, col):
    
    return df[col].resample('M', label='left', loffset='15d').mean()


In [ ]:
#Function that creates a plot object:
def plot_lpj_var(station, var, veg_type, lpj_var_dict, veg_type_to_lpjpath_dict, fluxnet_dict, monthly_values):
    
    #Compute monthly values
    if(monthly_values):
        
        #Read lpj-data for selected variable to a pandas dataframe:
        fluxnet_df = calc_monthly_values(read_fluxnet_dd(path_fluxnet_dd, station).set_index('TIMESTAMP'), fluxnet_dict[var])
        read_pnv_df = read_lpjguess_st('pnv', var, station).set_index('time')
        read_pnv_df.sort_index(inplace=True)
        pnv_df = calc_monthly_values(read_pnv_df, 'Value')
        read_veg_df = read_lpjguess_st(veg_type_to_lpjpath_dict[veg_type], var, station).set_index('time')
        read_veg_df.sort_index(inplace=True)
        veg_df = calc_monthly_values(read_veg_df, 'Value')
        
        
        
    else:
        fluxnet_df = read_fluxnet_dd(path_fluxnet_dd, station).set_index('TIMESTAMP')[fluxnet_dict[var]]
        pnv_df = read_lpjguess_st('pnv', var, station).set_index('time').Value
        pnv_df.sort_index(inplace=True)
        veg_df = read_lpjguess_st(veg_type_to_lpjpath_dict[veg_type], var, station).set_index('time').Value
        veg_df.sort_index(inplace=True)
        
    if(var=='NEE'):
        
        #Extract "var" data from every vegetation-type dataframe:
        x1 = fluxnet_df.index.values
        y1 = fluxnet_df.values
        x2 = pnv_df.index.values
        y2 = pnv_df.values*1000*(-1)
        x3 = veg_df.index.values
        y3 = veg_df.values*1000*(-1)
            
    else:
        #Extract "var" data from every vegetation-type dataframe:
        x1 = fluxnet_df.index.values
        y1 = fluxnet_df.values
        x2 = pnv_df.index.values
        y2 = pnv_df.values*1000
        x3 = veg_df.index.values
        y3 = veg_df.values*1000

    #Create a figure object:
    p = figure(plot_width = 900,
               plot_height = 500,
               x_axis_label = 'Time',
               y_axis_label = var +' ('+variables_dd_dict[fluxnet_dict[var]][1]+')',
               x_axis_type = 'datetime',
               title = var + ' per Vegetation Type ('+stations_dict[station][0]+', '+stations_dict[station][1]+')',
               min_border_top= 100)

    #Create an empty list that will store the legend info:
    legend_it = []

    #Add circle/line glyphs
    r0 = p.circle(x1, y1, radius=.12, color='DodgerBlue')
    r1 = p.circle(x2, y2, radius=.12, color='firebrick')
    r2 = p.circle(x3, y3, radius=.12, color='gold')
    
    l0 = p.line(x1, y1, line_width=2, color='DodgerBlue', name='FLUXNET Observations')
    l1 = p.line(x2, y2, line_width=2, color='firebrick', name='LPJ - PNV')
    l2 = p.line(x3, y3, line_width=2, color='gold', name='LPJ - '+veg_type)

    #Add the name and glyph info (i.e. colour and marker type) to the legend:
    legend_it.append((l0.name, [r0, l0]))
    legend_it.append((l1.name, [r1, l1]))
    legend_it.append((l2.name, [r2, l2]))

     #Create legend:
    legend = Legend(items=legend_it, location= 'bottom_center')
    legend.orientation = 'horizontal'
    legend.click_policy='hide'
    legend.spacing = 10 #sets the distance between legend entries

    #Add tooltip on hover:
    p.add_tools(HoverTool(tooltips=[('Type','$name'),
                                    ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
                                    (var,'@y{0.000f}'),
            ],
            formatters={
                '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
                },
            # display a tooltip whenever the cursor is vertically in line with a glyph
            mode='vline'
            ))    

    #Set title attributes:
    p.title.align = 'center'
    p.title.text_font_size = '13pt'
    p.title.offset = 15

    #Set axis label font style:
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units

    #Add legend to figure:
    p.add_layout(legend, 'below')

    #Deactivate hover-tool, which is by default active:
    p.toolbar.active_inspect = None

    #Return plot:
    return p

In [ ]:
def lpj_var_dropdown(lpj_var_dict):
    
    #Create dropdown list for stations:
    stations = Dropdown(options=sorted([(i[1][0], [i[0], i[1][0], i[1][1], i[1][2], i[1][3], i[1][4], i[1][5]])
                                        for i in stations_dict.items()]))
    
    #Create checkbox for monthly values:
    checkbx = Checkbox(value=False, description='Monthly values', disabled=False, indent=False)
    
    #Function that updates plot:
    def update_func(Station, Checkbox):
        
        #Call function to plot NEE:
        nee_plot = plot_lpj_var(Station[0], 'NEE', Station[4], lpj_var_dict, veg_type_to_lpjpath_dict, fluxnet_dict, Checkbox)
        
        #Call function to plot GPP:
        gpp_plot = plot_lpj_var(Station[0], 'GPP', Station[4], lpj_var_dict, veg_type_to_lpjpath_dict, fluxnet_dict, Checkbox)
        
        #Call function to plot RTOT:
        rtot_plot = plot_lpj_var(Station[0], 'RTOT', Station[4], lpj_var_dict, veg_type_to_lpjpath_dict, fluxnet_dict, Checkbox)
        
        
        #Show plots:
        show(column(nee_plot, gpp_plot, rtot_plot))
        
        
    
    #Create widget form:
    interact_c = interact_manual(update_func,
                                 Station=stations,
                                 Checkbox = checkbx)
    
    #Format widgets:
    interact_c.widget.children[0].layout.width = '430px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 200px'
    interact_c.widget.children[1].layout.width = '430px'
    interact_c.widget.children[1].layout.margin = '2px 2px 2px 287px'
    interact_c.widget.children[2].description = 'Update Plots'
    interact_c.widget.children[2].button_style = 'danger'
    interact_c.widget.children[2].style.button_color = '#3973ac'
    interact_c.widget.children[2].layout.margin = '10px 10px 40px 390px' # top/right/bottom/left
    

<div style="text-align: right"><a href="#introduction">Back to top</a></div>
<a id='lpjguess_map'></a>

## LPJ-GUESS Map data



In [ ]:
# define the geographical extend of all maps
map_latmin=35.0
map_latmax=71.0
map_lonmin=-10.0
map_lonmax=35.0

In [ ]:
# version to read all netcdf files because we want to extract timeseries for 2015-2018
# need to adjust fluxes
# 

def read_LPJ_fluxmap_2():

    time = []
    nee = []
    gpp = []
    rtot = []
    first = True
    for year in range(2015,2018+1):
        for month in range(1,12+1):
            filename=path_lpjmaps+'/lpj_'+str(year)+str(month).zfill(2)+'.nc4'
            ncfile = cdf.Dataset(filename)
            #print(ncfile)
            #print(ncfile.variables)
            if first:
                #print(ncfile)
                lon = ncfile.variables['lons'][:]
                lat = ncfile.variables['lats'][:]
                cell_area = ncfile.variables['area'][:,:]
                flux_units = ncfile.variables['nee'].units
                #print(flux_units)
                description = ncfile.Notes
                #print(description)
                first = False
            for m in range(ncfile.dimensions['nt'].size):
                time.append(cdf.num2date(ncfile.variables['times'][m],units=ncfile.variables['times'].units))
                nee.append(ncfile.variables['nee'][m,:,:]*1000.)
                gpp.append(ncfile.variables['gpp'][m,:,:]*1000.)
                rtot.append(ncfile.variables['rtot'][m,:,:]*-1.*1000.)    
            ncfile.close()

    nee = np.array(nee,dtype='float32')
    gpp = np.array(gpp,dtype='float32')
    rtot = np.array(rtot,dtype='float32')
            
    return lon, lat, time, nee, gpp, rtot, cell_area, flux_units, description

In [ ]:
#Function that reads a LPJ-netcdf for given year and month:
def read_LPJ_fluxmap(filename):
    
    """
    Project:      EUROCOM
    Created:      Fri Apr 13 09:00:00 2018
    Last Changed: Fri Feb 13 10:00:00 2020
    Version:      1.1.0
    Author(s):    Ute Karstens, Karolina Pantazatou
    
    Definition:   Function that takes a string as input, signifying the path to a netCDF-datafile,
                  reads the netcdf and returns its content.
    
    Input:        1. Full path to netCDF file (data type: String)
    
    
    Output:
                  1. a list of longitudes
                  2. a list of latitudes
                  3. a list of time instances (i.e. Python datetime objects)
                  4. a 3-dimensional array with NEE values (one month time period)
                  5. a 3-dimensional array with GPP values
                  6. a 3-dimensional array with RESPIRATION values 
                  7. cell_area, flux_units, description
    """
    
    #Declare and initialize lists:
    time = [] #store sequence of timepoints
    nee = []  #store nee-arrays
    gpp = []  #store
    rtot = []
    
    #Open netcdf-file with data:
    ncfile = cdf.Dataset(filename)
    
    #Get longitudes and latitudes:
    lon = ncfile.variables['lons'][:]
    lat = ncfile.variables['lats'][:]
    
    #Get the cell-area of ever grid-cell:
    cell_area = ncfile.variables['area'][:,:]
    
    #Get NEE flux units:
    flux_units = ncfile.variables['nee'].units
    
    #Get file description:
    description = ncfile.Notes

    #Loop through all timepoints:        
    for m in range(ncfile.dimensions['nt'].size):
        
        #Get time, nee, gpp and rtot values for every timepoint:
        time.append(cdf.num2date(ncfile.variables['times'][m],units=ncfile.variables['times'].units))
        nee.append(ncfile.variables['nee'][m,:,:])
        gpp.append(ncfile.variables['gpp'][m,:,:])
        rtot.append(ncfile.variables['rtot'][m,:,:])    
    
    #Close netcdf-file:
    ncfile.close()
    
    #Convert nee/gpp/rtot lists to numpy arrays:
    nee = np.array(nee,dtype='float32')
    gpp = np.array(gpp,dtype='float32')
    rtot = np.array(rtot,dtype='float32')
    
    #Return output:
    return lon, lat, time, nee, gpp, rtot, cell_area, flux_units, description

In [ ]:
#Function that creates a grid based on a list or numpy array of lat/lon values:
def get_mesh(lon,lat):
    
    """
    Project:      EUROCOM
    Created:      Fri Apr 13 09:00:00 2018
    Last Changed: Fri Apr 13 09:00:00 2018
    Version:      1.0.0
    Author(s):    Ute Karstens
    
    Definition:   Function that takes two list of longitudes and latitudes as input and
                  and returns NumPy meshgrids of longitudes and latitudes.
    
    Input:        1. Longitudes (data type: List or NumPy array of ints or floats)
                  2. Latitudes  (data type: List or NumPy array of ints or floats)
    
    
    Output:
                  1. Meshgrid of longitudes (data type: Numpy array of ints or floats) 
                  2. Meshgrid of latitudes  (data type: Numpy array of ints or floats) 
                  
    """
    
    #Shift the longitudes and latitudes:
    dlat2=abs(lat[2]-lat[1])/2.
    plat=np.append((lat-dlat2),(lat[-1]+(dlat2)))
    dlon2=abs(lon[2]-lon[1])/2.
    plon=np.append((lon-dlon2),(lon[-1]+(dlon2)))
    
    #Create meshgrid:
    xx, yy = np.meshgrid(plon, plat)
    
    #Return meshgrid:
    return xx,yy

In [ ]:
#Function that plots a single LPJ-GUESS map for a given timepoint:
def plot_map(fig, xx,yy,zz,nr,nc,iplot,title,vmin=-3,vmax=3,units='gC/m2/d',cb_name='PiYG_r',extend='both'):
    
    
#    'PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu',
#            'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic']
    
    
    #Set scale for features from Natural Earth:
    NEscale = '50m'    
    
    #Create a feature for Countries at 1:50m from Natural Earth:
    countries = cfeature.NaturalEarthFeature(category='cultural',
                                             name='admin_0_countries',
                                             scale=NEscale,facecolor='none')
    
    #Create a feature for Lakes at 1.50m from Natural Earth:
    lakes = cfeature.NaturalEarthFeature(category='physical', name='lakes', scale=NEscale, facecolor='none')
    
    #Set up a map:
    ax = fig.add_subplot(nr+1, nc, iplot, projection=ccrs.PlateCarree())
    
    #Define the spatial extent of the map (min/max lat/lon):
    img_extent = [xx.min(), xx.max(), yy.min(), yy.max()]
    
    #Set the spatial extent and the coordinate system:
    ax.set_extent(img_extent, crs=ccrs.PlateCarree())
    
    #Add gridlines:
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False, linewidth=0.3, color='gray', alpha=0.5)
    
    #Set gridline-parameters (spatial extent (min/max values of lat/lon) and interval(here=10))
    gl.ylocator = mticker.FixedLocator(np.arange(-90.,90.,10.))
    gl.xlocator = mticker.FixedLocator(np.arange(-180.,180.,10.))
    
    #Add Natural Earth countries:
    ax.add_feature(countries, edgecolor='black', linewidth=0.3)
    
    #Add Natural Earth lakes:
    ax.add_feature(lakes, edgecolor='black', linewidth=0.3)
    
    #Add raster with values:
    
    im = ax.pcolormesh(xx,yy,zz,cmap=cb_name,vmin=vmin,vmax=vmax)
    
    #Add colorbar:
    cbar=plt.colorbar(im,orientation='horizontal',pad=0.03,fraction=0.055,extend=extend)
    
    
    #Define colorbar parameters:
    cbar.outline.set_linewidth(1.0)
    cbar.ax.tick_params(labelsize=10) 
    cbar.set_label(units,fontsize=10)
    
    #Add plot title:
    ax.set_title(title,fontsize=14)
    
    #Add explanatory text under the colorbar (raster dataset min value):
    ax.text(0.01, -0.27, 'min: %.2f' % np.nanmin(zz),
            horizontalalignment='left',transform=ax.transAxes,fontsize=11)
    
    #Add explanatory text under the colorbar (raster dataset max value):
    ax.text(0.99, -0.27, 'max: %.2f' % np.nanmax(zz),
            horizontalalignment='right',transform=ax.transAxes,fontsize=11)
    

In [ ]:
def check_dates(y, m, d):
    
    """
    Project:      ClimBEco Research School - 2020
    Created:      Fri Feb 13 10:00:00 2020
    Last Changed: Fri Feb 13 10:00:00 2020
    Version:      1.0.0
    Author(s):    Karolina Pantazatou
    
    Definition:   Function that takes 3 strings as input parameters (i.e. year, month, day)
                  controls if the date is valid and returns a boolean value.
    
    Input:        1. Year (data type: String)
                  2. Month (data type: String)
                  3. Day (data type: String)
    
    
    Output:
                  1. True/False (data type: Boolean)
    """
    
    
    #Define & initialize help variable:
    check = False
   
    #Control for February (Leap Year and normal year):
    if(((int(y)%4==0) & (int(m)==2) & (int(d)<=29)) | ((int(m)==2) & (int(d)<=28))):
        
        #Set help-variable to "true":
        check = True
    
    
    #Control for months with 31 days (Jan, Mar, May, Jul, Aug, Oct, Dec):
    elif((int(m) in [1, 3, 5, 7, 8, 10, 12]) & (int(d)<32)):
        
        #Set help-variable to "true":
        check = True
        
        
    #Control for months with 30 days (Apr, Jun, Sep, Nov):
    elif((int(m) in [4, 6, 9, 11]) & (int(d)<31)):
        
        #Set help-variable to "true":
        check = True
    
    
    #If none above the above conditions is true:
    else:
        
        #Set help-variable to "false":
        check = False
    
    
    #Return boolean variable:
    return check
        
        
    

In [ ]:
def checkpathtofile(pathtofile):
    
    """
    Project:      ClimBEco Research School - 2020
    Created:      Fri Feb 13 10:00:00 2020
    Last Changed: Fri Feb 13 10:00:00 2020
    Version:      1.0.0
    Author(s):    Karolina Pantazatou
    
    Definition:   Function that checks if a file exists in given directory.
                  It takes a string, containing the full path to a datafile, as input and 
                  controls if that filename exists. It returns a boolean value as output.
                  If the file exists, it returns "True", otherwise "False".
    
    Input:        1. Full path to datafile (data type: String)
                  
    Output:
                  1. True/False (data type: Boolean)
    """

    #Control if path to file exists:
    if(os.path.exists(pathtofile)):
        return True
    
    else:
        return False
        
        #Print error-message:
        print("\033[0;31;1m"+'File not found...'+"\033[0;31;0m\n\n")
        
        
    

In [ ]:
#Function that creates a widget-form:
def create_widget_form_multi_maps():
    
    """
    Project:      ClimBEco Research School - 2020
    Created:      Fri Feb 13 15:00:00 2020
    Last Changed: Fri Feb 13 15:00:00 2020
    Version:      1.0.0
    Author(s):    Karolina Pantazatou
    
    Definition:   Function that creates a form of widgets to select the year,
                  the month, the day and the time (hour) for which the user
                  wishes to get the corresponding NEE, GPP and RESPIRATION 
                  maps. The user imay select 2 different timepoints and
                  visually compare the maps.
    
    Input:        No input parameters
                  
    Output:       Widget form
    
    """
    
        
    #Set style for widget descriptions:
    style = {'description_width': 'initial'}

    #Create dropdown-lists for time variables (year, month, day and hour) timepoint 1:
    years1 = Dropdown(options=['2015', '2016', '2017', '2018'],
                      value='2015',
                      description='Year:',
                      disabled=False)

    months1 = Dropdown(options=[str(i) for i in list(np.arange(1,13,1))],
                       value='1',
                       description='Month:',
                       disabled=False)

    days1 = Dropdown(options=[str(i) for i in list(np.arange(1,32,1))],
                     value='1',
                     description='Day:',
                     disabled=False)

    hours1 = Dropdown(options=['03:00', '09:00', '15:00', '21:00'],
                      value='03:00',
                      description='Hour:',
                      disabled=False)
    
    #Add FloatText-widgets for setting the colorbar limits:
    cbar_min1 = FloatText(value=-3.0,
                          description='Colorbar (min value):',
                          style=style,
                          disabled=False)

    cbar_max1= FloatText(value=3.0,
                         description='Colorbar (max value):',
                         style=style,
                         disabled=False)
    
    
    #Create dropdown-lists for time variables (year, month, day and hour) timepoint 2:
    years2 = Dropdown(options=['2015', '2016', '2017', '2018'],
                      value='2015',
                      description='Year:',
                      disabled=False)

    months2 = Dropdown(options=[str(i) for i in list(np.arange(1,13,1))],
                       value='1',
                       description='Month:',
                       disabled=False)

    days2 = Dropdown(options=[str(i) for i in list(np.arange(1,32,1))],
                     value='1',
                     description='Day:',
                     disabled=False)

    hours2 = Dropdown(options=['03:00', '09:00', '15:00', '21:00'],
                      value='03:00',
                      description='Hour:',
                      disabled=False)
    
    #Add FloatText-widgets for setting the colorbar limits:
    cbar_min2 = FloatText(value=-3.0,
                          description='Colorbar (min value):',
                          style=style,
                          disabled=False)

    cbar_max2= FloatText(value=3.0,
                         description='Colorbar (max value):',
                         style=style,
                         disabled=False)


    #Create a Button-widget to control execution:
    update_button = Button(description='Run',
                           disabled=False,
                           button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
                           tooltip='Click me')
    
    
    header_time_1 = Output()
    with header_time_1:
        display(HTML('<p style="font-size:13px;font-weight:bold;color:#6495ed;">Select values for 1st timepoint: </p>'))

    header_time_2 = Output()
    with header_time_2:
        display(HTML('<p style="font-size:13px;font-weight:bold;color:#6495ed;">Select values for 2nd timepoint: </p>'))

    
    

    #Add the time-related dropdown-widgets to a HBox:
    time_hbox1 = HBox([years1, months1, days1, hours1])
    
    #Add the colorbar-limit-related widgets to a HBox:
    cbar_hbox1 = HBox([cbar_min1, cbar_max1])
    
    #Add time1-header, time_hbox1 & cbar_hbox1 to a VBox:
    time_vbox1 = VBox([header_time_1, time_hbox1, cbar_hbox1])
    
    #Add the time-related dropdown-widgets to a HBox:
    time_hbox2 = HBox([years2, months2, days2, hours2])
    
    #Add the colorbar-limit-related widgets to a HBox:
    cbar_hbox2 = HBox([cbar_min2, cbar_max2])
    
    #Add time2-header, time_hbox2 & cbar_hbox2 to a VBox:
    time_vbox2 = VBox([header_time_2, time_hbox2, cbar_hbox2])

    #Add all widgets to a VBox:
    form = VBox([time_vbox1, time_vbox2, update_button])

    #Set font of all widgets in the form:
    update_button.style.button_color = '#1f78b4'


    #Initialize form output:
    form_out = Output()

    #Initialize results output:
    results_out = Output()


    #Define update function:
    def update_plot(button_c):

        #Control input:
        control_dates1 = check_dates(years1.value, months1.value, days1.value)
        control_dates2 = check_dates(years2.value, months2.value, days2.value)
        
        
        #Create path to files:        
        filename1=path_lpjmaps+'/lpj_'+years1.value+months1.value.zfill(2)+'.nc4'
        filename2=path_lpjmaps+'/lpj_'+years2.value+months2.value.zfill(2)+'.nc4'
        
        #Check if file exists in given path:
        control_file1 = checkpathtofile(filename1)
        control_file2 = checkpathtofile(filename2)
        

        #If the input is correct:
        if(control_dates1 & control_dates2 & control_file1 & control_file2):
            
            
            #Get data from netcdf-file:
            lon_LPJ1, lat_LPJ1, time_LPJ1, nee_LPJ_map1, gpp_LPJ_map1, rtot_LPJ_map1, cell_area_LPJ1, flux_units_LPJ1, description_LPJ1 = read_LPJ_fluxmap(filename1)
            lon_LPJ2, lat_LPJ2, time_LPJ2, nee_LPJ_map2, gpp_LPJ_map2, rtot_LPJ_map2, cell_area_LPJ2, flux_units_LPJ2, description_LPJ2 = read_LPJ_fluxmap(filename2)
            
            #Get index number for selected date:
            i1 = [i for i in range(len(time_LPJ1))
                 if time_LPJ1[i]==datetime.strptime(years1.value+'/'+months1.value+'/'+days1.value+' '+hours1.value, '%Y/%m/%d %H:%M' )]
            
            #Get index number for selected date:
            i2 = [i for i in range(len(time_LPJ2))
                 if time_LPJ2[i]==datetime.strptime(years2.value+'/'+months2.value+'/'+days2.value+' '+hours2.value, '%Y/%m/%d %H:%M' )]
            
            #Check that data exist for given timepoint:
            if((len(i1)>0) & (len(i2)>0)):
                
                #Convert latitude/longitude lists to meshgrid:
                xx_LPJ1,yy_LPJ1 = get_mesh(lon_LPJ1,lat_LPJ1)
                xx_LPJ2,yy_LPJ2 = get_mesh(lon_LPJ2,lat_LPJ2)

                #Change units:
                fact = 1000.*1000.*4.*(12./44.)

                #Set unit labels:
                units_bio='gC/m2/d'
                cb_name_bio='PiYG_r'

                #Create titles:
                time_title_1=time_LPJ1[i1[0]].strftime("%Y-%m-%d %H:%M") #1st timepoint
                time_title_2=time_LPJ2[i2[0]].strftime("%Y-%m-%d %H:%M") #2nd timepoint
                
                nee_title_1='LPJ-GUESS NEE'+'   '+time_title_1 #1st timepoint
                nee_title_2='LPJ-GUESS NEE'+'   '+time_title_2 #2nd timepoint
                
                gpp_title_1='LPJ-GUESS GPP'+'   '+time_title_1 #1st timepoint
                gpp_title_2='LPJ-GUESS GPP'+'   '+time_title_2 #2nd timepoint
                
                rtot_title_1='LPJ-GUESS RTOT'+'   '+time_title_1 #1st timepoint
                rtot_title_2='LPJ-GUESS RTOT'+'   '+time_title_2 #2nd timepoint

                #Change units of raster dataset:
                zz_nee_1 = nee_LPJ_map1[i1[0],:,:] * fact / cell_area_LPJ1[:,:]
                zz_gpp_1 = gpp_LPJ_map1[i1[0],:,:] * fact / cell_area_LPJ1[:,:]
                zz_rtot_1 = (rtot_LPJ_map1[i1[0],:,:] * fact / cell_area_LPJ1[:,:])*(-1)
                zz_nee_2 = nee_LPJ_map2[i2[0],:,:] * fact / cell_area_LPJ2[:,:]
                zz_gpp_2 = gpp_LPJ_map2[i2[0],:,:] * fact / cell_area_LPJ2[:,:]
                zz_rtot_2 = (rtot_LPJ_map2[i2[0],:,:] * fact / cell_area_LPJ2[:,:])*(-1)

                
                #Display selection:
                with results_out:

                    #Clear previous results:
                    clear_output()
                    
                    #Plot maps:
                    fig = plt.figure(figsize=(18,20))
                    plot_map(fig,xx_LPJ1,yy_LPJ1,zz_nee_1,2,3,1,nee_title_1,vmin=cbar_min1.value,vmax=cbar_max1.value,units=units_bio,cb_name=cb_name_bio)
                    plot_map(fig,xx_LPJ1,yy_LPJ1,zz_gpp_1,2,3,2,gpp_title_1,vmin=cbar_min1.value,vmax=cbar_max1.value,units=units_bio,cb_name=cb_name_bio)
                    plot_map(fig,xx_LPJ1,yy_LPJ1,zz_rtot_1,2,3,3,rtot_title_1,vmin=cbar_min1.value,vmax=cbar_max1.value,units=units_bio,cb_name=cb_name_bio)
                    
                    plot_map(fig,xx_LPJ2,yy_LPJ2,zz_nee_2,2,3,4,nee_title_2,vmin=cbar_min2.value,vmax=cbar_max2.value,units=units_bio,cb_name=cb_name_bio)
                    plot_map(fig,xx_LPJ2,yy_LPJ2,zz_gpp_2,2,3,5,gpp_title_2,vmin=cbar_min2.value,vmax=cbar_max2.value,units=units_bio,cb_name=cb_name_bio)
                    plot_map(fig,xx_LPJ2,yy_LPJ2,zz_rtot_2,2,3,6,rtot_title_2,vmin=cbar_min2.value,vmax=cbar_max2.value,units=units_bio,cb_name=cb_name_bio)
                    
                    plt.show()
                    plt.close()

            #No data for given timepoint: 
            else:

                #Display selection:
                with results_out:

                    #Clear previous results:
                    clear_output()

                    #Print error-message:
                    print("\033[0;31;1m"+'No data available for given timepoint...\nTry again!'+"\033[0;31;0m\n\n")


        #Invalid input values: 
        else:

            #Display selection:
            with results_out:

                #Clear previous results:
                clear_output()

                #Print error-message:
                print("\033[0;31;1m"+'Invalid date...\nTry again!'+"\033[0;31;0m\n\n")



    #Call update-function when button is clicked:
    update_button.on_click(update_plot)
    update_button.layout.margin = '50px 100px 40px 400px' #top, right, bottom, left
    time_vbox1.layout.margin = '25px 0px 0px 0px'
    time_vbox2.layout.margin = '50px 0px 0px 0px'
    cbar_hbox1.layout.margin = '20px 0px 0px 203px'
    cbar_max1.layout.margin = '0px 0px 0px 160px'
    cbar_min1.layout.width='180px'
    cbar_max1.layout.width='180px'
    cbar_hbox2.layout.margin = '20px 0px 0px 203px'
    cbar_max2.layout.margin = '0px 0px 0px 160px'
    cbar_min2.layout.width='180px'
    cbar_max2.layout.width='180px'

    #Open form object:
    with form_out:

        #Clear previous selections in form:
        clear_output()

        #Display form and results:
        display(form, results_out)


    #Display form:
    display(form_out)  

<div style="text-align: right"><a href="#introduction">Back to top</a></div>
<a id='lumia'></a>

## LUMIA


In [ ]:
def read_EUROCOM_drought_fluxmap(filename):

    import cdo 
    cdo = cdo.Cdo()    
        
    ncfile = cdf.Dataset(filename)

    #print(ncfile)
    #print(ncfile.variables)
    
    lon = ncfile.variables['longitude'][:]
    lat = ncfile.variables['latitude'][:]
    time = cdf.num2date(ncfile.variables['time'][:],units=ncfile.variables['time'].units)+timedelta(14)
    bio_prior = ncfile.variables['fnee_prior'][:,:,:]
    bio_posterior = ncfile.variables['fnee_post'][:,:,:]
    oce = ncfile.variables['focn'][:,:,:]
    fossil = ncfile.variables['fff'][:,:,:]
    flux_units = ncfile.variables['fnee_prior'].units
    #print(flux_units)
    description = ncfile.comment
    #print(description)
    
    ncfile.close()
    
    # get cell area using cdo commands
    cell_area = cdo.gridarea(input=filename,  returnArray = 'cell_area') 

    # restrict to common region

    ind_lat=np.where((lat >= map_latmin) & (lat <= map_latmax))
    ind_lon=np.where((lon >= map_lonmin) & (lon <= map_lonmax))
    lon_min=np.min(ind_lon)
    lon_max=np.max(ind_lon)
    lat_min=np.min(ind_lat)
    lat_max=np.max(ind_lat)
    lat=lat[lat_min:lat_max+1]
    lon=lon[lon_min:lon_max+1]
    bio_prior = bio_prior[:,lat_min:lat_max+1,lon_min:lon_max+1]
    bio_posterior = bio_posterior[:,lat_min:lat_max+1,lon_min:lon_max+1]
    oce = oce[:,lat_min:lat_max+1,lon_min:lon_max+1]
    fossil = fossil[:,lat_min:lat_max+1,lon_min:lon_max+1]
    cell_area = cell_area[lat_min:lat_max+1,lon_min:lon_max+1]
    
    # should we restrict to 2015-2018 here?
    
    return lon, lat, time, bio_prior, bio_posterior, oce, fossil, cell_area, flux_units, description


In [ ]:
#Function that creates a widget-form:
def create_widget_form_multi_maps_lumia():
    
    """
    Project:      ClimBEco Research School - 2020
    Created:      Fri Feb 13 15:00:00 2020
    Last Changed: Fri Feb 13 15:00:00 2020
    Version:      1.0.0
    Author(s):    Karolina Pantazatou
    
    Definition:   Function that creates a form of widgets to select the year,
                  the month, the day and the time (hour) for which the user
                  wishes to get the corresponding NEE, GPP and RESPIRATION 
                  maps. The user imay select 2 different timepoints and
                  visually compare the maps.
    
    Input:        No input parameters
                  
    Output:       Widget form
    
    """
    
        
    #Set style for widget descriptions:
    style = {'description_width': 'initial'}

    #Create dropdown-lists for time variables (year, month, day and hour) timepoint 1:
    years1 = Dropdown(options=[2015, 2016, 2017, 2018],
                      value=2015,
                      description='Year:',
                      disabled=False)

    months1 = Dropdown(options=[i for i in list(np.arange(1,13,1))],
                       value=1,
                       description='Month:',
                       disabled=False)

    
    #Add FloatText-widgets for setting the colorbar limits:
    cbar_min1 = FloatText(value=-3.0,
                          description='Colorbar (min value):',
                          style=style,
                          disabled=False)

    cbar_max1= FloatText(value=3.0,
                         description='Colorbar (max value):',
                         style=style,
                         disabled=False)
    
    
    #Create dropdown-lists for time variables (year, month, day and hour) timepoint 2:
    years2 = Dropdown(options=[2015, 2016, 2017, 2018],
                      value=2015,
                      description='Year:',
                      disabled=False)

    months2 = Dropdown(options=[i for i in list(np.arange(1,13,1))],
                       value=1,
                       description='Month:',
                       disabled=False)
    
    #Add FloatText-widgets for setting the colorbar limits:
    cbar_min2 = FloatText(value=-3.0,
                          description='Colorbar (min value):',
                          style=style,
                          disabled=False)

    cbar_max2= FloatText(value=3.0,
                         description='Colorbar (max value):',
                         style=style,
                         disabled=False)


    #Create a Button-widget to control execution:
    update_button = Button(description='Run',
                           disabled=False,
                           button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
                           tooltip='Click me')
    
    
    header_time_1 = Output()
    with header_time_1:
        display(HTML('<p style="font-size:13px;font-weight:bold;color:#6495ed;">Select values for 1st timepoint: </p>'))

    header_time_2 = Output()
    with header_time_2:
        display(HTML('<p style="font-size:13px;font-weight:bold;color:#6495ed;">Select values for 2nd timepoint: </p>'))

    
    

    #Add the time-related dropdown-widgets to a HBox:
    time_hbox1 = HBox([years1, months1])
    
    #Add the colorbar-limit-related widgets to a HBox:
    cbar_hbox1 = HBox([cbar_min1, cbar_max1])
    
    #Add time1-header, time_hbox1 & cbar_hbox1 to a VBox:
    time_vbox1 = VBox([header_time_1, time_hbox1, cbar_hbox1])
    
    #Add the time-related dropdown-widgets to a HBox:
    time_hbox2 = HBox([years2, months2])
    
    #Add the colorbar-limit-related widgets to a HBox:
    cbar_hbox2 = HBox([cbar_min2, cbar_max2])
    
    #Add time2-header, time_hbox2 & cbar_hbox2 to a VBox:
    time_vbox2 = VBox([header_time_2, time_hbox2, cbar_hbox2])

    #Add all widgets to a VBox:
    form = VBox([time_vbox1, time_vbox2, update_button])

    #Set font of all widgets in the form:
    update_button.style.button_color = '#1f78b4'


    #Initialize form output:
    form_out = Output()

    #Initialize results output:
    results_out = Output()


    #Define update function:
    def update_plot(button_c):
        
        
        #Create path to files:        
        filename1=path_lumia+'co2flux_monthly_lumia_core_2009_2018.nc'

        
        #Check if file exists in given path:
        control_file1 = checkpathtofile(filename1)
        

        #If the input is correct:
        if(control_file1):
            
            
            #Get data from netcdf-file:
            lon_LUMIA,lat_LUMIA,time_LUMIA,bio_prior_LUMIA_map,bio_poste_LUMIA_map,oce_LUMIA_map,ff_LUMIA_map,cell_area_LUMIA_map,flux_units_LUMIA,comment = read_EUROCOM_drought_fluxmap(filename1)
            
            #Get index number for selected date:
            i1 = [n for n in range(len(time_LUMIA)) if ((time_LUMIA[n].year==years1.value)& (time_LUMIA[n].month==months1.value))]
            i2 = [m for m in range(len(time_LUMIA)) if ((time_LUMIA[m].year==years2.value)& (time_LUMIA[m].month==months2.value))]
            
            #Check that data exist for given timepoint:
            if((len(i1)>0) & (len(i2)>0)):
                
                #Convert latitude/longitude lists to meshgrid:
                xx_LUMIA,yy_LUMIA = get_mesh(lon_LUMIA,lat_LUMIA)

                #Change units:
                fact = 1000.*24.

                #Set unit labels:
                units_bio='gC/m2/d'
                cb_name_bio='PiYG_r'

                #Create titles:
                time_title_1=time_LUMIA[i1[0]].strftime("%Y-%m-%d %H:%M") #1st timepoint
                time_title_2=time_LUMIA[i2[0]].strftime("%Y-%m-%d %H:%M") #2nd timepoint
                
                prior_title_1='LUMIA PRIOR'+'   '+time_title_1 #1st timepoint
                prior_title_2='LUMIA PRIOR'+'   '+time_title_2 #2nd timepoint
                
                post_title_1='LUMIA POSTERIOR'+'   '+time_title_1 #1st timepoint
                post_title_2='LUMIA POSTERIOR'+'   '+time_title_2 #2nd timepoint
                
                diff_title_1='LUMIA DIFF'+'   '+time_title_1 #1st timepoint
                diff_title_2='LUMIA DIFF'+'   '+time_title_2 #2nd timepoint

                #Change units of raster dataset:
                zz_prior_1 = bio_prior_LUMIA_map[i1[0],:,:] * fact
                zz_post_1 = bio_poste_LUMIA_map[i1[0],:,:] * fact
                zz_diff_1 = (bio_prior_LUMIA_map[i1[0],:,:]- bio_poste_LUMIA_map[i1[0],:,:])* fact
                zz_prior_2 = bio_prior_LUMIA_map[i2[0],:,:] * fact
                zz_post_2 = bio_poste_LUMIA_map[i2[0],:,:] * fact
                zz_diff_2 = (bio_prior_LUMIA_map[i2[0],:,:]- bio_poste_LUMIA_map[i2[0],:,:])* fact

                
                #Display selection:
                with results_out:

                    #Clear previous results:
                    clear_output()
                    
                    #Plot maps:
                    fig = plt.figure(figsize=(18,20))
                    plot_map(fig,xx_LUMIA,yy_LUMIA,zz_prior_1,2,3,1,prior_title_1,vmin=cbar_min1.value,vmax=cbar_max1.value,units=units_bio,cb_name=cb_name_bio)
                    plot_map(fig,xx_LUMIA,yy_LUMIA,zz_post_1,2,3,2,post_title_1,vmin=cbar_min1.value,vmax=cbar_max1.value,units=units_bio,cb_name=cb_name_bio)
                    plot_map(fig,xx_LUMIA,yy_LUMIA,zz_diff_1,2,3,3,diff_title_1,vmin=cbar_min1.value,vmax=cbar_max1.value,units=units_bio,cb_name=cb_name_bio)
                    
                    plot_map(fig,xx_LUMIA,yy_LUMIA,zz_prior_2,2,3,4,prior_title_2,vmin=cbar_min2.value,vmax=cbar_max2.value,units=units_bio,cb_name=cb_name_bio)
                    plot_map(fig,xx_LUMIA,yy_LUMIA,zz_post_2,2,3,5,post_title_2,vmin=cbar_min2.value,vmax=cbar_max2.value,units=units_bio,cb_name=cb_name_bio)
                    plot_map(fig,xx_LUMIA,yy_LUMIA,zz_diff_2,2,3,6,diff_title_2,vmin=cbar_min2.value,vmax=cbar_max2.value,units=units_bio,cb_name=cb_name_bio)
                    
                    plt.show()
                    plt.close()

            #No data for given timepoint: 
            else:

                #Display selection:
                with results_out:

                    #Clear previous results:
                    clear_output()

                    #Print error-message:
                    print("\033[0;31;1m"+'No data available for given timepoint...\nTry again!'+"\033[0;31;0m\n\n")


        #Invalid input values: 
        else:

            #Display selection:
            with results_out:

                #Clear previous results:
                clear_output()

                #Print error-message:
                print("\033[0;31;1m"+'Invalid date...\nTry again!'+"\033[0;31;0m\n\n")



    #Call update-function when button is clicked:
    update_button.on_click(update_plot)
    update_button.layout.margin = '50px 100px 40px 270px' #top, right, bottom, left
    time_vbox1.layout.margin = '25px 0px 0px 0px'
    time_vbox2.layout.margin = '50px 0px 0px 0px'
    cbar_hbox1.layout.margin = '20px 0px 0px 120px'
    cbar_max1.layout.margin = '0px 0px 0px 120px'
    cbar_min1.layout.width='180px'
    cbar_max1.layout.width='180px'
    cbar_hbox2.layout.margin = '20px 0px 0px 120px'
    cbar_max2.layout.margin = '0px 0px 0px 120px'
    cbar_min2.layout.width='180px'
    cbar_max2.layout.width='180px'
    form.layout.margin = '0px 0px 0px 100px'

    #Open form object:
    with form_out:

        #Clear previous selections in form:
        clear_output()

        #Display form and results:
        display(form, results_out)


    #Display form:
    display(form_out)  

<div style="text-align: right"><a href="#introduction">Back to top</a></div><a id='tsmaps'></a>

## Extract time series from maps

In [ ]:
# function to convert station longitude and latitude (slat, slon) to indices of STILT model grid (ix,jy)
def lonlat_2_ixjy(slon,slat,mlon,mlat):
    #slon, slat: longitude and latitude of station
    #mlon, mlat: 1-dim. longitude and latitude of model grid
    ix = (np.abs(mlon-slon)).argmin()
    jy = (np.abs(mlat-slat)).argmin()
    return ix,jy

In [ ]:
#Function that converts a list of cftime-objects to
#a list of datetime-objects:
def cftime_to_datetime(cftime_ls):
    
    #Convert list of cftime objects to list of datetime objects:
    return [pd.to_datetime(str(cftime_ls[i].year) + '/'+ str(cftime_ls[i].month)+ '/'+ str(cftime_ls[i].day) +' '+ str(cftime_ls[i].hour)+':'+str(cftime_ls[i].minute)+':'+str(cftime_ls[i].second))
            for i in range(len(cftime_ls))]
    
    

<div style="text-align: right"> <a href="#introduction">Back to top</a></div><a id='edgar'></a>

## EDGAR

In [ ]:
# read edgar anthropogenic emissions on specified grid resolution
def read_edgar(path_edgar,resolution):
    
    str_resolution = '{:03.1f}'.format(resolution)
    #print('Anthropogenic emissions on '+str_resolution+'x'+str_resolution+' grid')
    filename = path_edgar+'v432_CO2_TOT_C_2010.'+str_resolution+'x'+str_resolution+'.europe.nc'
    f_edgar = cdf.Dataset(filename,'r')
    #print(f_edgar)
    lat = f_edgar.variables['lat'][:]
    lon = f_edgar.variables['lon'][:]
    emis = f_edgar.variables['emi_co2'][:,:]
    flux_units = f_edgar.variables['emi_co2'].units
    #print(flux_units)
    f_edgar.close()

    ind_lat=np.where((lat >= map_latmin) & (lat <= map_latmax))
    ind_lon=np.where((lon >= map_lonmin) & (lon <= map_lonmax))
    lon_min=np.min(ind_lon)
    lon_max=np.max(ind_lon)
    lat_min=np.min(ind_lat)
    lat_max=np.max(ind_lat)
    lat=lat[lat_min:lat_max+1]
    lon=lon[lon_min:lon_max+1]
    emis = emis[lat_min:lat_max+1,lon_min:lon_max+1]

    return lon, lat, emis, flux_units

<div style="text-align: right"> <a href="#introduction">Back to top</a></div>
<a id='masks'></a>

<br>
<br>

## Country masks


In [ ]:
#read country masks
def read_country_masks(filename_countrymask, map_latmin=35.0, map_latmax=71.0, map_lonmin=-10.0, map_lonmax=35.0):
    
    #Read netCDF:
    f_MASK = cdf.Dataset(filename_countrymask,'r')
   
    #Get lists with latitude and longitude values:
    lat=f_MASK.variables['latitude'][:]
    lon=f_MASK.variables['longitude'][:]
    
    #Get country mask multidimensional array:
    countryMask=f_MASK.variables['country_mask'][:,:,:]
    
    #Get list of country codes:
    countryCode=cdf.chartostring(f_MASK.variables['country_code'][:,:])
    
    #Get list of country names:
    countryName=cdf.chartostring(f_MASK.variables['country_name'][:,:])
    
    #Close netCDF:
    f_MASK.close()
    
    #Get an array of index-values for latitudes inside the study area:
    ind_lat=np.where((lat >= map_latmin) & (lat <= map_latmax))
    
    #Get an array of index-values for longitudes inside the study area:
    ind_lon=np.where((lon >= map_lonmin) & (lon <= map_lonmax))
    
    #Get min/max longitudes and latitudes:
    lon_min=np.min(ind_lon)
    lon_max=np.max(ind_lon)
    lat_min=np.min(ind_lat)
    lat_max=np.max(ind_lat)
    
    #Extract latitudes and longitudes that are within the study area:
    lat=lat[lat_min:lat_max+1]
    lon=lon[lon_min:lon_max+1]
    
    #Clip country-mask to the defined spatial extent:
    countryMask = countryMask[:,lat_min:lat_max+1,lon_min:lon_max+1]

    #Return values:
    return lon, lat, countryMask, countryCode, countryName 

In [ ]:
# Get the name of the countries included in the country-mask netCDF:
def get_cmask_cname(filename_countrymask):
    
    #Read netCDF:
    f_MASK = cdf.Dataset(filename_countrymask,'r')
    
    #Get list of country names:
    countryName=cdf.chartostring(f_MASK.variables['country_name'][:,:])
    
    #Close netCDF:
    f_MASK.close()

    #Return values:
    return countryName 

In [ ]:
# read cell area
def read_cellarea(filename_cellarea, map_latmin=35.0, map_latmax=71.0, map_lonmin=-10.0, map_lonmax=35.0):
    
    #Read netCDF:    
    f_area = cdf.Dataset(path_masks +filename_cellarea,'r')
    #print(f_area)
    #Get lists with latitude and longitude values:
    lat=f_area.variables['lat'][:]
    lon=f_area.variables['lon'][:]
    
    #Get cell area multidimensional array:
    cellarea=f_area.variables['cell_area'][:,:]
        
    #Close netCDF:
    f_area.close()
    
    #Get an array of index-values for latitudes inside the study area:
    ind_lat=np.where((lat >= map_latmin) & (lat <= map_latmax))
    
    #Get an array of index-values for longitudes inside the study area:
    ind_lon=np.where((lon >= map_lonmin) & (lon <= map_lonmax))
    
    #Get min/max longitudes and latitudes:
    lon_min=np.min(ind_lon)
    lon_max=np.max(ind_lon)
    lat_min=np.min(ind_lat)
    lat_max=np.max(ind_lat)
    
    #Extract latitudes and longitudes that are within the study area:
    lat=lat[lat_min:lat_max+1]
    lon=lon[lon_min:lon_max+1]
    
    #Clip cellarea to the defined spatial extent:
    cellarea = cellarea[lat_min:lat_max+1,lon_min:lon_max+1]

    #Return values:
    return lon, lat, cellarea

<br>

### Country Masks - Read Country Boundaries as arrays of lats and longs

In [ ]:
#Create function that returns a pandas dataframe of country boundaries in lat/lon:
def country_boundaries_lat_lon(path):

    #Import functions:
    import csv
    import codecs
    import gzip
    import xml.etree.cElementTree as et
    from os.path import dirname, join

    #Set NaN:
    nan = float('NaN')

    #Create a dictionary to store lats/lons per country:
    data = {}

    #Open csv with country-boundary info:
    with gzip.open(path) as f:

        #Set encoding to utf-8:
        decoded = codecs.iterdecode(f, "utf-8")
        next(decoded)

        #Read csv-file:
        reader = csv.reader(decoded, delimiter=',', quotechar='"')

        #Loop through every row in the csv-file:
        for row in reader:

            #Get the geometry, the country code and the country name:
            geometry, code, name = row

            #Get geometry:
            xml = et.fromstring(geometry)

            #Create lists to store tuples of lats and lons for current country:
            lats = []
            lons = []

            #Loop through geometry coordinates:
            for i, poly in enumerate(xml.findall('.//outerBoundaryIs/LinearRing/coordinates')):

                #Get coordinates for all points in a country polygon (geometry):
                coords = (c.split(',')[:2] for c in poly.text.split())

                #Split lats from lons for every point coordinate:
                lats, lons = list(zip(*[(float(lat), float(lon)) for lon, lat in
                    coords]))

                #Populate dictionary with country name and tuples of latitudes and longitudes:
                data[code + str(i)] = {
                    'name'   : name,
                    'lats'   : lats,
                    'lons'   : lons,
                }

    #Convert dictionary to Pandas DataFrame:
    worldmap = pd.DataFrame.from_dict(data, orient='index')

    #Return dataframe:
    return worldmap

<br>

### Country Masks - Create a grid for Bokeh

In [ ]:
def get_grid_lats_lons(max_lat, min_lat, max_lon, min_lon, step):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue Feb 26 17:27:00 2020
    Last Changed:     Tue Feb 26 17:27:00 2020
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes five numeric variables as input and returns
                      two lists of lists with longitudes and latitudes(representing a
                      grid). The output is used for plotting a grid in Bokeh. 
                      
    Input:            1. Max latitude of grid (var_name: 'max_lat', var_type: Float)
                      2. Min latitude of grid (var_name: 'min_lat', var_type: Float)
                      3. Max longitude of grid (var_name: 'max_lon', var_type: Float)
                      4. Min longitude of grid (var_name: 'min_lon', var_type: Float)
                      5. Grid resolution (var_name: 'step', var_type: Float)
    
    Output:           1. list of lists of longitudes (var_name: 'xx', var_type: Float)
                      2. list of lists of latitudes (var_name: 'yy', var_type: Float)

    """

    #Import modules:
    import numpy as np

    #Create a numpy array of values ranging from min-longitude
    #to max-longitude with a predefined step:
    xx_arr = np.arange(min_lon, max_lon + step, step)

    #Create a numpy array of values ranging from min-latitude
    #to max-latitude with a predefined step:
    yy_arr = np.arange(min_lat, max_lat + step, step)

    #Round floats to one decimal degree:
    xx_row = [round(xx_arr[k],1) for k in range(len(xx_arr))]
    yy_row = [round(yy_arr[l],1) for l in range(len(yy_arr))]

    #Create list of longitudes:
    xx = [xx_row for k in range(len(yy_row))]

    #Create list of latitudes:
    yy = [[y for j in range(len(xx_row))] for y in yy_row]
    
    #Return lists:
    return xx, yy

<br>

### Country Masks - Plot Country Mask


In [ ]:
#Function that plots a cmask:
def plot_cmask(countries, cmask, cmask_type):
    
    #Convert geopandas dataframe to geojson:
    countries_source = GeoJSONDataSource(geojson=countries.to_json())
    
    #Create plot object:
    p = figure(width=900, height=500, x_axis_type=None, y_axis_type=None,
               x_range=[-10,35], y_range=[35,70])
    
    #Add country-layer:
    p.patches('xs','ys', source = countries_source, line_color = 'black', line_width = 0.25, fill_color=None)
    
    #Add country-mask:
    p.image_rgba(image=[cmask[cmask_type]],
                 x=[-10.0], y=[35],
                 dw=[45.0], dh=[36.0],
                 name='image')

    #Set x-axis layout parameters:
    xaxis = LinearAxis()   
    p.add_layout(xaxis, 'below')
    
    #Set y-axis layout parameters:
    yaxis = LinearAxis()
    p.add_layout(yaxis, 'left')
    
    #Add Bokeh Grid:
    p.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
    p.add_layout(Grid(dimension=1, ticker=yaxis.ticker))
    
    #Create hover-tool:
    p.add_tools(HoverTool(names=["image"],
                          tooltips=[("cell partition", "@image{0.00}"),
                                    ("lat", "$y"),
                                    ("lon", "$x")]))
    
    #Deactivate hover-tool, which is by default active:
    p.toolbar.active_inspect = None

    #Show plot:
    show(p)

<br>

### Country Masks - Widget Form

In [ ]:
#Create widget-form:
def cmask_widget_form():
    
    #Get shapefile from Natural Earth (10m resolution) with country boundaries:
    shapefile = path_geo_natural_earth10m+'ne_10m_admin_0_countries.shp'
    
    #Read shapefile to Geopandas DataFrame:
    gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'CONTINENT','geometry']]
    
    #Extract polygons for European countries and Turkey:
    countries_map = gdf.loc[(gdf.CONTINENT=="Europe") | (gdf.ADMIN=="Turkey")| (gdf.ADMIN=="Cyprus")]

    #Rename columns:
    countries_map.columns = ['country', 'country_code', 'continent','geometry']
    
    
    #Get the country name for all EEZ country-masks:
    eez_cname_array = get_cmask_cname(path_masks + 'eurocomCountryMaskEEZ_0.5x0.5.nc')

    #Create a list of tuples like ('Sweden', 47) --- >(cname, ind_num) for EEZ cmasks
    eez_cname_ls = [(eez_cname_array[i], i) for i in range(len(eez_cname_array))]

    #Get the country name for all country-masks:
    cname_array = get_cmask_cname(path_masks  + 'eurocomCountryMask_0.5x0.5.nc')

    #Create a list of tuples like ('Sweden', 47) --- >(cname, ind_num)
    cname_ls = [(cname_array[i], i) for i in range(len(cname_array))]

    cnames_ls = sorted([(cname_ls[i][0], [eez_cname_ls[j][1], i])
                 for i in range(len(cname_ls))
                 for j in range(len(eez_cname_ls))
                 if cname_ls[i][0]==eez_cname_ls[j][0]])





    #Add dropdown list for the selection of country:
    countries = Dropdown(options=cnames_ls,
                         layout={'width': 'max-content'},
                         description='Country:',
                         style={'description_width': 'initial'},
                         disabled=False)

    #Add dropdown list for the selection of country mask:
    cmasks = Dropdown(options=['No EEZ', 'EEZ'],
                      layout={'width': 'max-content'},
                      description='Country mask:',
                      style={'description_width': 'initial'},
                      disabled=False)

    #Add dropdown for selection of different grid resolutions:
    cresolutions = Dropdown(options=['0.1', '0.5'],
                            #layout={'width': 'max-content'},
                            description='Mask-resolution (in degrees):',
                            style={'description_width': 'initial'},
                            disabled=False)

    #Create a Button-widget to control execution:
    exe_button = Button(description='Run',
                        disabled=False,
                        button_style='danger',
                        tooltip='Click me')


    #Add the widgets for "cmasks", countries" and "country mask resolutions" to a VBox:
    country_box = VBox([cmasks, countries, cresolutions])

    #Store country_box and button in a VBox (final widget-form):
    form = VBox([country_box, exe_button])

    #Initialize form output:
    form_out = Output()

    #Initialize results output:
    results_out = Output()


    #Define update function:
    def update_func(button_c):


        #Check choice of country mask type:
        if(cmasks.value == 'EEZ'):

            #Construct path to cmask:
            pathtocmask=path_masks + 'eurocomCountryMaskEEZ_'+cresolutions.value+'x'+cresolutions.value+'.nc'

            #Get country index:
            c_ind = countries.value[0]


        #if cmaks.value=='No EEZ'    
        else:

            #Construct path to cmask:
            pathtocmask=path_masks + 'eurocomCountryMask_'+cresolutions.value+'x'+cresolutions.value+'.nc'

            #Get country index:
            c_ind = countries.value[1]


        #Get cmask-data from netcdf:
        lon, lat, cMask, cCode, cName = read_country_masks(pathtocmask)


        #Open output object:
        with results_out:

            #Clear previous results:
            clear_output()     

            #plot_cmask(worldmap, cMask, countries.value)
            plot_cmask(countries_map, cMask, c_ind)


    #Call update-function when button is clicked:
    exe_button.on_click(update_func)

    #Set font of all widgets in the form:
    exe_button.style.button_color = '#1f78b4'
    exe_button.layout.margin = '50px 100px 40px 290px' #top, right, bottom, left
    country_box.layout.margin = '25px 0px 0px 65px'
    form.layout.margin = '25px 0px 0px 0px'
    countries.layout.margin = '0px 0px 0px 36px'
    countries.layout.width = '300px'
    cmasks.layout.width = '335px'
    cmasks.layout.margin = '0px 0px 5px 85px'
    countries.layout.margin = '0px 0px 5px 120px'
    cresolutions.layout.width = '418px'
    form.layout.margin = '0px 0px 0px 110px'

    
    #Open form object:
    with form_out:

        #Clear previous selections in form:
        clear_output()

        #Display form and results:
        display(form, results_out)


    #Display form:
    display(form_out)

In [ ]:
def multi_table(table_list, table_names=[]):
    """ Accept a list of Pandas Dataframes                 
        returns an HTML table where the data frames are side by side
        if table_names are provided add additional row with names
    """
    if len(table_list)==len(table_names):
        th = '<tr>'+ ''.join(['<th style="text-align:left; border:1px solid grey">' + name + '</th>' for name in table_names]) + '</tr>'        
    else:
        th = ''
                
    return HTML(
        '<table style="border:1px solid black"><tr style="background-color:white">' + th +
        ''.join(['<td style="border:1px solid grey">' + table.to_html() + '</td>' for table in table_list]) +
        '</tr></table>'
    )

In [ ]:
def getEmission(country, unit):
    e = []
    e.append(df_grid01[unit][df_grid01['c_code'] == country].iloc[0])
    e.append(df_grid01EEZ[unit][df_grid01EEZ['c_code'] == country].iloc[0])
    e.append(df_grid05[unit][df_grid05['c_code'] == country].iloc[0])
    e.append(df_grid05EEZ[unit][df_grid05EEZ['c_code'] == country].iloc[0])
    return e 


In [ ]:
def getEmissions(df, country, unit):
    e = []
    for d in df:
        e.append(d[unit][d['c_code'] == country].iloc[0])
    return e 


<a id='stclass'></a>


# Station class

In [ ]:
class Station():
    
    '''
    Description: Class that represents 
    
    '''
    
    def __init__(self, station_code, eco_var_name=''):
        
        self._code = station_code
        self._eco_var = eco_var_name
        self.info = None
        self._latitude = None
        self._longitude = None
        self._lpj = None
        self._lpjmap = None
        self._fluxnet = None
        self._lumia = None      
        
        self.read_info()
    
    def read_info(self):
        self.info = stations_dict[self.code]
    
    @property
    def code(self):
        return self._code
    
    @property
    def eco_var_name(self):
        return self._eco_var
    
    @eco_var_name.setter
    def eco_var_name(self, eco_var_name):
        self._eco_var=eco_var_name 
    
    @property
    def lat(self):
        return self.info[4]
    
    @property
    def lon(self):
        return self.info[5]  
    
    @property
    def lpj(self):
        return self._lpj
    
    @lpj.setter
    def lpj(self, data):
        self._lpj=data
     
    @property
    def fluxnet(self):
        return self._fluxnet
    
    @fluxnet.setter
    def fluxnet(self, data):
        self._fluxnet=data
        
    @property
    def lpjmap(self):
        return self._lpjmap
    
    @lpjmap.setter
    def lpjmap(self, data):
        self._lpjmap=data
    
    @property
    def lumia(self):
        return self._lumia
    
    @lumia.setter
    def lumia(self, data):
        self._lumia=data
    

<br>
<br>
<br>
<br>
<img src="../ancillarydata/logos/sponsors_climbeco_course_2020.PNG" width="1000" align="left"/>
<br>
<br>